In [322]:
import random
import numpy as  np
import math

#range=[0,4] i.e 5 sized grid
size=5
size-=1
nvehicle=100                 #no. of vehicles on the simulation


def fit(particle,target):    #fitness fucnction
    return 1/(1+math.sqrt(abs(target[0]-particle[0])**2+abs(target[1]-particle[1])**2))

def init_vehicles():         #to create vehicles
    global nvehicle
    veh=[]
    for _ in range(nvehicle):
        i=random.randint(0,size+1)
        j=random.randint(0,size+1)
        x=random.randint(0,size+1)
        y=random.randint(0,size+1)
        if i==size+1:
            i,j=0,0
        elif j==size+1:
            i,j=2,size
        if x==size+1:
            x,y=0,0
        elif y==size+1:
            x,y=2,size
        while (i,j)==(x,y) :
            j=random.randint(0,size)
        veh.append(((i,j),(x,y),_))
    return veh

        
def init_positions(veh):        #initialise vehpos to make it usable for simulation
    vehpos=[]
    for v in veh:
        vehpos.append((list(v[0]),v[1],v[2]))
    return vehpos


def init_lanes():               #create lanes and intersections(2d)
    lanes=[[[-1],[-1],[-1],[-1]] for _ in range((size+1)*(size+1))]
    return lanes



def spawn_vehicles(lanes,vehpos):          #spawn vehicles onto map
    for vehicle in vehpos:
        x,y=vehicle[1][0]-vehicle[0][0],vehicle[1][1]-vehicle[0][1]
        h,v=1,1
        if x<0:
            v=-1
        if y<0:
            h=-1
        ch=math.sqrt(abs(x))/(math.sqrt(abs(x))+math.sqrt(abs(y)))
        if random.random()<=ch:
            choice=0
            vehicle[0][0]+=v
            if v==1:
                lanes[vehicle[0][0]*size+vehicle[0][1]][2].append(vehicle[2])
            else:
                lanes[vehicle[0][0]*size+vehicle[0][1]][0].append(vehicle[2])
        else:
            choice=1
            vehicle[0][1]+=h
            if h==1:
                lanes[vehicle[0][0]*size+vehicle[0][1]][3].append(vehicle[2])
            else:
                lanes[vehicle[0][0]*size+vehicle[0][1]][1].append(vehicle[2])


def execute(lanes,signal,vehpos):       #simulation
    count,cost=0,100
    global nvehicle
    for _ in range(500):
        for i in range(len(lanes)):
            signal[i][0]-=1
            if signal[i][0]==0:
                signal[i][1]+=1
                signal[i][1]%=4
                signal[i][0]=signal[i][2+signal[i][1]]
            if len(lanes[i][signal[i][1]])>1:
                vh=lanes[i][signal[i][1]][1]
                lanes[i][signal[i][1]].pop(1)
                x,y=vehpos[vh][1][0]-vehpos[vh][0][0],vehpos[vh][1][1]-vehpos[vh][0][1]
                if x!=0 or y!=0:                                 #x for vertical, y for horizontal
                    h,v=1,1
                    if x<0:
                        v=-1
                    if y<0:
                        h=-1
                    ch=math.sqrt(abs(x))/(math.sqrt(abs(x))+math.sqrt(abs(y)))
                    if random.random()<=ch:
                        vehpos[vh][0][0]+=v
                        if v==1:
                            lanes[vehpos[vh][0][0]*(size+1)+vehpos[vh][0][0]][2].append(vh)
                        else:
                            lanes[vehpos[vh][0][0]*(size+1)+vehpos[vh][0][0]][0].append(vh)
                    else:
                        vehpos[vh][0][1]+=h
                        if h==1:
                            lanes[vehpos[vh][0][0]*(size+1)+vehpos[vh][0][0]][3].append(vh)
                        else:
                            lanes[vehpos[vh][0][0]*(size+1)+vehpos[vh][0][0]][1].append(vh)
                else:
                    count+=1
        cost+=(nvehicle-count)
        if count==nvehicle:
            return [count,cost]
                        


In [323]:
def regular_signals():             #signal timing without preference for a lane
    signal=[]
    for _ in range((size+1)*(size+1)):
        signal.append([1,0,5,5,5,5])
    return signal
        


In [324]:
def find_path(particles,target):  #finding and saving path of best particle for a vehicle(PSO)
    count=0
    path=[]
    best=[0,0,0]
    for particle in particles:
        cur=[(particle[0],particle[1])]
        if particle[0]+particle[2]>size or particle[0]+particle[2]<0:
            particle[2]=0
        if particle[1]+particle[3]>size or particle[1]+particle[3]<0:
            particle[3]=0
        particle[0]+=particle[2]
        particle[1]+=particle[3]
        f=fit(particle,target)
        if f>best[0]:
            best[:]=f,particle[0],particle[1]
        cur.append((particle[0],particle[1]))
        path.append(cur)
    found=4
    cbest=[0,0,0]
    x=40
    while found==4:
        #print('iteration')
        best=cbest
        for i in range(4):
            xp,yp=target[0]-particles[i][0],target[1]-particles[i][1]
            chk=math.sqrt(xp**2+yp**2)
            if chk>1:
                n=1/chk
                xp,yp=xp*n,yp*n
            xg,yg=best[1]-particles[i][0],best[2]-particles[i][1]
            chk=math.sqrt(xg**2+yg**2)
            if chk>1:
                n=1/chk
                xg,yg=xg*n,yg*n
            xres,yres=particles[i][2]+xp+xg,particles[i][3]+yp+yg
            chk=math.sqrt(xres**2+yres**2)
            if chk>1:
                n=1/chk
                xres,yres=xres*n,yres*n
            particles[i][2],particles[i][3]=xres,yres
            if particles[i][0]+particles[i][2]>size or particles[i][0]+particles[i][2]<0:
                particles[i][2]=0
            if particles[i][1]+particles[i][3]>size or particles[i][1]+particles[i][3]<0:
                particles[i][3]=0
            particles[i][0],particles[i][1]=particle[0]+particle[2],particle[1]+particle[3]
            pbest=[particles[i][0]+xp,particles[i][1]+yp]
            f=fit(pbest,target)
            if f>cbest[0]:
                cbest[1:]=particles[i][:2]
                cbest[0]=f
            f=fit(particles[i],target)
            path[i].append((particles[i][0],particles[i][1]))
            if f>=0.9:
                found=i
        best=cbest
        x-=1
        if x==0 :
            break
    if found==4:
        count+=1
        return [(0,0),(0,0)]
    return path[found]
    


def pso_path():                   #Creating paricles for pso
    boid_paths=[]
    for v in veh:
        path_search_particles=[]
        i,j=v[0]
        #print(v)
        for _ in range(4):
            x=random.random()-0.5
            y=random.random()-0.5
            r=math.sqrt(x**2+y**2)
            n=1/r
            search_particle=[i,j,x*n,y*n]
            path_search_particles.append(search_particle)
        path=find_path(path_search_particles,v[1])
        boid_paths.append(path)
    return boid_paths


def modify_lanes(boid_paths):          #setting preference of lanes based on particle paths
    mod_lanes=np.ones(((size+1)*(size+1),4))
    for boid_path in boid_paths:
        tmp_w=np.zeros(((size+1)*(size+1),4))
        for z in range(len(boid_path)-1):
            x,y=boid_path[z+1][0]-boid_path[z][0],boid_path[z+1][1]-boid_path[z][1]
            for i in range(size+1):
                for j in range(size+1):
                    f=fit(boid_path[z],[i,j])
                    if f>0.25 and f!=1:
                        v=0
                        if x>0:
                            v=2
                        tmp_w[i*size+j][v]=max(tmp_w[i*(size+1)+j][v],abs(x)*(f**2))
                        h=1
                        if x>0:
                            h=3
                        tmp_w[i*size+j][h]=max(tmp_w[i*(size+1)+j][h],abs(y)*(f**2))
        mod_lanes+=tmp_w
    for i in range(len(mod_lanes)):
        x=sum(mod_lanes[i])
        for j in range(4):
            mod_lanes[i][j]=(mod_lanes[i][j]*4/x)
    return mod_lanes


def modified_signals(mod_lanes):         #setting the preference onto controller
    signal=[]
    for _ in range((size+1)*(size+1)):
        signal.append([1,0,1+round(mod_lanes[_][0]),round(1+mod_lanes[_][1]),round(1+mod_lanes[_][2]),round(1+mod_lanes[_][3])])
    return signal

In [352]:
veh=init_vehicles()
rcost=0
mcost=0

for _ in range(10):
    vehpos=init_positions(veh)
    lanes=init_lanes()
    signal=regular_signals()
    spawn_vehicles(lanes,vehpos)

    count,cost1=execute(lanes,signal,vehpos)    
    
    
    
    vehpos=init_positions(veh)
    boid_paths=pso_path()
    mod_lanes=modify_lanes(boid_paths)
    signal=modified_signals(mod_lanes)
    spawn_vehicles(lanes,vehpos)

    count,cost2=execute(lanes,signal,vehpos)
    rcost+=cost1
    mcost+=cost2

#print(mcost,rcost)
print('The average traffic using PSO is % 3.2f%c of the former'%(mcost/rcost*100,'%'))

The average traffic using PSO is  94.84% of the former
